In [ ]:
pip install -U google-generativeai 
pip install openai 

import google.generativeai as genai #importa gemini
import openai #importa openai
import requests #necesario
import os #necesario

In [ ]:
openai.api_key = "" #apikey de openia
genai.configure(api_key="") #apikey de gemini

In [ ]:
#en el system_instruction se da el contexto
model = genai.GenerativeModel(
    modelName="gemini-1.5-flash",
    generationConfig=generation_config,
    system_instruction="Eres un manager de fútbol un profesional como Pep Guardiola el hombre que más sabe de fútbol, y recomendará a los mejores jugadores para la posición del campo solicitada, dando datos como su valor, edad y habilidad con el balón. Además generas una imagen de el jugador con otro modelo por lo que si el usuario te dice no después de la primera recomendación es por que no requirió la imagen.",
)

#modelo de google
generationConfig = {
    "temperature": 0.5,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}


In [ ]:
historyChat = []
# funcion para generar la imagen desde openai
def generateImageOpenai(prompt):
    responseImage = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    # retorna la imagen en una url
    response = responseImage['data'][0]['url']
    return response


def chat():
    chatSession = model.start_chat(history=historyChat)
    
    print("Soy Pep Guardiola un entrenador de fútbol. Doy recomendaciones de jugadors que puedes necesitar para tu equipo, adelante pideme la posición del campo en la que te interesa fichar a un jugador.")
    
    while True:

        mensajeUsuario = input("Tu: ")

        if mensajeUsuario.lower() == "salir":  # Salir del chat
            break

        response = chatSession.send_message(mensajeUsuario)
        print(response.text)  

        # Actualizar historial con la respuesta de Gemini
        historyChat.append({'role': 'user', 'parts': [mensajeUsuario]})
        historyChat.append({'role': 'model', 'parts': [response.text]})

        if len(historyChat) == 2:   
            
            imageGenerate = input("¿Deseas generar una imagen con OpenAI basada en tu mensaje? (sí/no): ").strip().lower()
            
            if imageGenerate == "sí" or imageGenerate == "si":
                image_url = generateImageOpenai(mensajeUsuario)
                print(f"Image URL: {image_url}")
            elif imageGenerate == "no":
                print("Elija otra posicion del campo o otra opcion de jugador, si desea salir escriba 'salir' ")

            else:
                print("Respuesta no válida. Por favor, responde con 'sí' o 'no'.")

                # Agregar la URL de la imagen al historial
                historyChat.append({'role': 'model', 'parts': [f"Image URL: {image_url}"]})

chat()